In [ ]:
!pip install geopandas folium ipywidgets requests

In [ ]:
import gdown

# Google Drive file IDs
DRIVE_IDS = {
    "addresses.csv": "1W0oqnxK26-EAsrSOBCt32wOonsA3LL8o",
    "schools.csv":   "1koPoihMbfQZWxBdfUMnkenUnMrruHie-"
}

for fname, fid in DRIVE_IDS.items():
    url = f"https://drive.google.com/uc?id={fid}"
    print(f"Downloading {fname}…")
    gdown.download(url, fname, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1W0oqnxK26-EAsrSOBCt32wOonsA3LL8o
From (redirected): https://drive.google.com/uc?id=1W0oqnxK26-EAsrSOBCt32wOonsA3LL8o&confirm=t&uuid=08d6c018-20ec-445f-aae5-4aa25ae88dd3
To: /content/addresses.csv
100%|██████████| 220M/220M [00:02<00:00, 95.8MB/s]


Downloading...
From: https://drive.google.com/uc?id=1koPoihMbfQZWxBdfUMnkenUnMrruHie-
To: /content/schools.csv
100%|██████████| 78.1k/78.1k [00:00<00:00, 51.4MB/s]


In [ ]:
# Cell 3 (unchanged)
import pandas as pd
import math

schools   = pd.read_csv("schools.csv",   sep=",")  # comma-delimited
addresses = pd.read_csv("addresses.csv", sep=";")  # semicolon-delimited


In [ ]:
# Cell 4 – Interactive UI & Folium map (default OSM)
import folium
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import files
import math

def haversine(lon1, lat1, lon2, lat2):
    R = 3959  # miles
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat/2)**2
         + math.cos(math.radians(lat1))
         * math.cos(math.radians(lat2))
         * math.sin(dlon/2)**2)
    return 2 * R * math.asin(math.sqrt(a))

# Widgets
school_dropdown = widgets.Dropdown(
    options=schools['label'].sort_values().tolist(),
    description='School:'
)
radius_slider = widgets.SelectionSlider(
    options=[0.1,0.2,0.3,0.4,0.5,0.6] + list(range(1,6)),
    value=0.5,
    description='Radius (mi):',
    orientation='horizontal',
    readout=True
)
run_button = widgets.Button(description='► Run Buffer')
output = widgets.Output()

def on_run(_):
    with output:
        output.clear_output()
        sel = school_dropdown.value
        row = schools[schools['label']==sel].iloc[0]
        slon, slat = row['lon'], row['lat']
        print(f"Buffering around {sel} at ({slat:.5f}, {slon:.5f}) — {radius_slider.value} mi")

        # compute & filter
        addresses['distance'] = addresses.apply(
            lambda r: haversine(slon, slat, r['lon'], r['lat']), axis=1)
        within = addresses[addresses['distance'] <= radius_slider.value]
        print(f"Found {len(within)} addresses")

        # build map with only default OSM tiles
        m = folium.Map(location=[slat, slon], zoom_start=14, tiles='OpenStreetMap')
        folium.Circle(
            [slat, slon],
            radius=radius_slider.value * 1609,
            color='blue', fill=False,
            popup=f"{sel} Buffer"
        ).add_to(m)
        for _, r in within.iterrows():
            folium.Marker(
                [r['lat'], r['lon']],
                icon=folium.Icon(color='red', icon='envelope'),
                popup=r['address']
            ).add_to(m)

        display(HTML(m._repr_html_()))

        # download CSV
        fname = f"{sel.replace(' ','_')}_{radius_slider.value}mi.csv"
        within[['address','lon','lat','distance']].to_csv(fname, index=False)
        files.download(fname)

run_button.on_click(on_run)
display(widgets.VBox([school_dropdown, radius_slider, run_button, output]))
